In [1]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import obb_login
from openbb import obb
from humbldata.core.utils.env import Env
import polars as pl

# obb_login()
obb.account.login(pat=Env().OBB_PAT, remember_me=True)

In [2]:
result = obb.equity.price.historical(symbol="SPY", start_date="1960-12-01", end_date="2025-01-01", provider="fmp")
result.to_df()

,open,high,low,close,volume,vwap,adj_close,unadjusted_volume,change,change_percent
date,,,,,,,,,,
1993-01-29,43.97,43.97,43.75,43.94,1003200,43.9075,24.46,1003200.0,-0.03080,-0.000682
1993-02-01,43.97,44.25,43.97,44.25,480500,44.1100,24.63,480500.0,0.28125,0.006368
1993-02-02,44.22,44.38,44.13,44.34,201300,44.2675,24.68,201300.0,0.12500,0.002714
1993-02-03,44.41,44.84,44.38,44.81,529400,44.6100,24.94,529400.0,0.40625,0.009007
1993-02-04,44.97,45.09,44.88,45.00,531500,44.9850,25.05,531500.0,0.03125,0.000667
...,...,...,...,...,...,...,...,...,...,...
2024-12-24,596.06,601.34,595.47,601.30,33160100,598.5425,599.50,33160100.0,5.24000,0.008791
2024-12-26,599.50,602.48,598.08,601.34,41338891,600.3500,599.54,41338891.0,1.84000,0.003069
2024-12-27,597.54,597.78,590.76,595.01,64969310,595.2725,593.23,64969310.0,-2.53000,-0.004234


In [3]:
result = obb.economy.calendar(provider='nasdaq', country='united_kingdom', start_date='2024-12-01', end_date='2025-01-01')
df = result.to_df()


OpenBBError: 
[Error] -> 1 validations error(s)
[Arg] provider -> input: nasdaq -> Input should be 'fmp' or 'tradingeconomics'

In [ ]:
df[df["event"].str.contains("CPI")]

In [ ]:
%load_ext autoreload
%autoreload

from humbldata.toolbox.toolbox_controller import Toolbox

toolbox = Toolbox(
    symbols="SPY,IWM,DIA,QQQ",
    interval="1d",
    start_date="2000-01-01",
    end_date="2024-12-03",
    # membership="anonymous"
)
toolbox

In [ ]:
# import nest_asyncio
# nest_asyncio.apply()

result = toolbox.technical.humbl_channel(historical=False, window="1mo", _boundary_group_down=False, chart=True)
result

In [ ]:
result.to_polars()

In [ ]:
result.command_params

In [ ]:
result.context_params

## Test `calc_up_down_pct`


In [ ]:
from humbldata.portfolio.analytics.watchlist.helpers import calc_up_down_pct


df = pl.DataFrame({
    "symbol": ["AAPL", "GOOGL", "MSFT"],
    "bottom_price": [5.18, 15.07, 16.24],
    "recent_price": [10.05, 20.31, 16.42],
    "top_price": [11.23, 25.17, 30.09],
})
result = calc_up_down_pct(df)
print(result)

# Setting up UserTable

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.standard_models.portfolio.analytics.watchlist_table import (
    WatchlistTableQueryParams,
)
from humbldata.core.utils.openbb_helpers import get_latest_price

symbols = WatchlistTableQueryParams(symbol="AAPL, MSFT, NVDA").symbol
symbols

# Tesing Async


## Get Sector Async

In [ ]:
%load_ext autoreload
%autoreload 2

import polars as pl

from humbldata.core.utils.openbb_helpers import (
    aget_equity_sector,
    aget_etf_category,
)

stocks = pl.Series([
    "GOOGL",  # Communication Services
    "AMZN",   # Consumer Discretionary
    "KO",     # Consumer Staples
    "XOM",    # Energy
    "JPM",    # Financials
    "JNJ",    # Health Care
    "HON",    # Industrials
    "AAPL",   # Information Technology
    "LIN",    # Materials
    "PLD",    # Real Estate
    "NEE"     # Utilities
])

# Define objects and lists for different symbol groups

# Equities
equity_symbols = ["AAPL", "NVDA", "TSLA"]

# ETFs
etf_symbols = ["XLE", "XLF", "XLU"]

# Mixed (equities and ETFs)
mixed_symbols = ["AAPL", "XLE", "XLU", "TSLA"]

# Edge cases
edge_symbols = ["BITI", "LNGG", "ETHU", "DBA"]


# test = (await aget_etf_category(symbols=["AAPL", "XLE", "DBA"], provider="yfinance")).collect()
test2 = (await aget_equity_sector(symbols=equity_symbols, provider="yfinance")).collect()
# test2 = (await aget_equity_sector(symbols=["XLE", "AAPL"], provider="yfinance")).collect()
test2

In [ ]:
symbols = ["CORN", 'SLV', 'DBA', "XLE", "AAPL", "FXE", "SPY", "QQQ", "BITO", "MAXI", "SATO", "BIL", "LQD", "QAI", "MNA", "CLSE"]
symbols2 = ["AAPL", "FXE", "SPY", "MAXI", "GLD"]
symbols3 = commodity_etf_symbols = ["GLD", "IAU", "SLV", "GLDM", "PDBC", "SGOL", "FTGC", "DBC", "SIVR", "USO", "IAUM", "GSG", "BCI", "PPLT", "COMT", "GLTR", "OUNZ", "BAR", "UNG", "DBA", "AAAU", "DJP", "CMDT", "KRBN", "PALL", "COM", "NBCM", "CMDY", "DBO", "BCD", "KCCA", "CPER", "HGER", "USCI", "DBB", "DBP", "GCC", "WEAT", "BNO", "UGA", "DGP", "IGLD", "COMB", "DBE", "FGDL", "CORN", "USL", "CCRV", "DJCB", "IAUF", "PLTM", "BDRY", "PDBA", "BGLD", "UCIB", "SOYB", "GRN", "PIT", "BCIM", "RENW", "SDCI", "UNL", "SHNY", "DCMT", "TAGS", "CANE", "TMET", "KEUA", "HCOM", "HARD", "EVMT", "USG", "AMPD", "DZZ", "DGZ", "KMET", "CMCI", "USOY", "ZSC", "TILL", "BWET", "DULL", "ZSB", "LNGG", "USOI", "OILK", "SLVO", "GLDI", "BOIL", "KOLD", "AGQ", "GLL", "UGL", "ZSL", "UCO", "SCO"]



df = (await aget_etf_category(symbols=edge_symbols)).collect()
df

In [ ]:
from humbldata.portfolio.analytics.watchlist.helpers import normalize_asset_class


normalized_df = normalize_asset_class(df).select(["symbol", "category"])
normalized_df

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import aget_etf_category


df2 = (await aget_etf_category(symbols=edge_symbols)).collect()
df2

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.portfolio.analytics.watchlist.helpers import aget_asset_class_filter

(await aget_asset_class_filter(symbols=["XLE", "XLF", "XLK"])).collect()

## Test aget_sector_filter

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.portfolio.analytics.watchlist.helpers import aget_sector_filter

etf_data = pl.LazyFrame({
    "symbol": ["BITI", "LNGG", "ETHU", "DBA"],
    "category": [
        "Trading--Miscellaneous",
        "Equity Energy",
        None,
        "Commodities Focused",
    ]
})

(await aget_sector_filter(symbols=mixed_symbols)).collect()

# Test user_table_engine

In [ ]:
from humbldata.portfolio.analytics.watchlist.model import watchlist_table_engine
import polars as pl

etf_data = [
    pl.DataFrame(
        {"symbol": ["AAPL", "NVDA", "TSLA"], "category": [None, None, None]}
    ),
    pl.DataFrame(
        {
            "symbol": ["XLE", "XLF", "XLU"],
            "category": ["Equity Energy", "Financial", "Utilities"],
        }
    ),
    pl.DataFrame(
        {
            "symbol": ["AAPL", "XLE", "XLU", "TSLA"],
            "category": [None, "Equity Energy", "Utilities", None],
        }
    ),
    pl.DataFrame(
        {
            "symbol": ["BITI", "LNGG", "ETHU", "DBA"],
            "category": [
                "Trading--Miscellaneous",
                "Equity Energy",
                None,
                "Commodities Focused",
            ],
        }
    ),
]

out: pl.LazyFrame = await watchlist_table_engine(symbols=edge_symbols)
out.collect()

## Test UserTableFetcher

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.portfolio.portfolio_controller import Portfolio

portfolio = Portfolio(symbols=["AAPL"], user_role="anonymous")
portfolio

In [ ]:
result = (await portfolio.analytics.watchlist_table())

In [ ]:
result.to_polars()

In [ ]:
result.to_polars()

# Testing Sandbox

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.toolbox.toolbox_controller import Toolbox
import datetime
from datetime import timedelta

# start_date = "2020-01-01"
# end_date = "2024-01-01"
start_date = (datetime.datetime.now() - datetime.timedelta(days=365))
end_date = datetime.date(2024, 7, 18)

toolbox = Toolbox(
    symbols=["AAPL", "AMD"],
    interval="1d",
    start_date="2022-07-18",
    end_date=end_date,
    membership="peon"
)

In [ ]:
result = toolbox.technical.humbl_channel(historical=False, window="1m", _boundary_group_down=False, chart=True)
result


In [ ]:
result.warnings[0]

In [ ]:
import pickle
from pathlib import Path

# Ensure the directory exists
Path("../tests/test_data").mkdir(parents=True, exist_ok=True)

# Save the result object to the specified file
with Path("../tests/test_data/mandelbrot_humblobject.pkl").open("wb") as file:
    pickle.dump(result, file)

print("Result object saved successfully.")
